In [8]:
import os

current_dir = os.getcwd()
root_dir = os.path.dirname(current_dir)
os.chdir(root_dir)
print("Current directory:", os.getcwd())

Current directory: c:\Users\Marcus Bergdahl\data_science_projects


In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier



In [2]:
import tensorflow as tf
print(tf.__version__)

2.10.1


In [3]:
import keras
keras.__version__

'2.10.0'

In [12]:
dont_use_feature = ["season_start_year","GW","id","team_h","team_a","train_score","home","away","kickoff_year","kickoff_month","kickoff_date"]

train_df=pd.read_csv('artifacts/train.csv').drop(dont_use_feature,axis=1)
test_df=pd.read_csv('artifacts/test.csv').drop(dont_use_feature,axis=1)
val_df=pd.read_csv('artifacts/validation.csv').drop(dont_use_feature,axis=1)

In [13]:
train_df

,label_1,label_X,label_2,rounds_left,win_share_latest_5_games_overall_home_team,draw_share_latest_5_games_overall_home_team,loss_share_latest_5_games_overall_home_team,avg_goals_scored_latest_5_games_overall_home_team,avg_goals_conceded_latest_5_games_overall_home_team,win_share_latest_5_games_overall_away_team,...,player_diff_max_f_rolling_mean_6_impact_game_threat,player_diff_max_f_rolling_mean_6_impact_game_transfers_balance,player_diff_max_f_rolling_mean_6_impact_game_hourly_rate_goals_scored,player_diff_max_f_rolling_mean_6_impact_game_hourly_rate_assists,player_diff_max_f_rolling_max_6_impact_game_influence,player_diff_max_f_rolling_max_6_impact_game_creativity,player_diff_max_f_rolling_max_6_impact_game_threat,player_diff_max_f_rolling_max_6_impact_game_transfers_balance,player_diff_max_f_rolling_max_6_impact_game_hourly_rate_goals_scored,player_diff_max_f_rolling_max_6_impact_game_hourly_rate_assists
0,0,1,0,22.0,0.2,0.0,0.8,0.8,1.8,0.4,...,0.424145,538.246856,0.010437,0.010406,1.393622,4.391184,-0.041120,1935.761004,0.001829,6.243496e-02
1,0,0,1,8.0,0.4,0.6,0.0,0.8,0.2,0.2,...,-0.237234,-962.402354,0.000010,0.000000,-0.069866,0.339786,-0.085648,3066.740891,0.000062,0.000000e+00
2,0,0,1,23.0,0.0,0.2,0.8,0.8,2.0,0.6,...,-1.791782,-4634.318948,-0.010132,-0.019630,0.838057,0.401216,0.547112,-17562.788734,0.000000,3.312291e-03
3,0,0,1,14.0,0.2,0.4,0.4,1.2,1.8,1.0,...,0.537508,7251.600616,0.000041,-0.010111,1.154472,0.065181,2.738412,71884.883721,0.000123,6.938894e-18
4,1,0,0,-5.0,0.0,0.2,0.8,0.4,1.8,0.0,...,0.281974,1.130054,0.010132,0.010132,4.569604,0.110660,0.049238,-1846.785306,0.060790,6.079027e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,1,0,0,-7.0,0.6,0.0,0.4,2.0,1.8,0.0,...,1.580998,31.626353,0.030405,0.010101,2.344523,-0.352484,5.819689,-2433.729582,0.060852,6.060606e-02
643,0,0,1,24.0,0.2,0.4,0.4,1.4,1.2,0.6,...,-0.871409,175.083477,0.010372,0.000000,2.950207,1.789588,-1.550152,462.753036,0.060790,0.000000e+00
644,0,0,1,23.0,0.4,0.0,0.6,1.6,1.8,0.6,...,-0.964750,-1304.099292,0.020233,-0.010173,-0.046227,-0.614925,-4.636364,-5387.099291,0.000123,-6.103764e-02
645,1,0,0,2.0,0.4,0.2,0.4,1.2,0.8,0.0,...,0.568503,1406.894837,-0.010050,-0.010132,-3.299906,-1.313070,1.581015,-3054.701851,-0.060300,-6.079027e-02


In [14]:
X_train = pd.concat([train_df, test_df]).drop(columns=['label_1', 'label_X', 'label_2'], axis=1).reset_index(drop=True)
y_train = pd.concat([train_df, test_df])[['label_1', 'label_X', 'label_2']].reset_index(drop=True)

X_val = val_df.drop(columns=['label_1', 'label_X', 'label_2'], axis=1).reset_index(drop=True)
y_val = val_df[['label_1', 'label_X', 'label_2']].reset_index(drop=True)

In [39]:
X_train.shape[1]

305

In [49]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.optimizers import Adam, SGD

# Define your deep learning model with increased complexity
def create_model(num_layers=3, units_per_layer=128, optimizer='adam', learning_rate=0.001):
    model = Sequential()
    model.add(Dense(512, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(Dropout(0.5))
    
    for _ in range(num_layers):
        model.add(Dense(units_per_layer, activation='relu'))
        model.add(Dropout(0.5))
    
    model.add(Dense(3, activation='softmax'))
    
    if optimizer == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Create KerasClassifier for GridSearchCV
model = KerasClassifier(build_fn=create_model, epochs=30, batch_size=32)

# Define the hyperparameters to tune
parameters = {
    'optimizer': ['adam', 'sgd'],
    'learning_rate': [0.001, 0.01, 0.1],
    'num_layers': [2, 3, 4],
    'units_per_layer': [64, 128, 256]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=model, param_grid=parameters, cv=3)

# Perform grid search on your training data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and the corresponding accuracy
print("Best Hyperparameters: ", grid_search.best_params_)
print("Best Accuracy: ", grid_search.best_score_)


Epoch 1/30


C:\Users\Marcus Bergdahl\AppData\Local\Temp\ipykernel_20500\3603085052.py:28: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs=30, batch_size=32)


18/18 [==============================] - 0s 2ms/step - loss: 17170.1152 - accuracy: 0.3339
Epoch 2/30
18/18 [==============================] - 0s 2ms/step - loss: 7619.6973 - accuracy: 0.3478
Epoch 3/30
18/18 [==============================] - 0s 2ms/step - loss: 2707.3708 - accuracy: 0.3722
Epoch 4/30
18/18 [==============================] - 0s 2ms/step - loss: 1043.0052 - accuracy: 0.4174
Epoch 5/30
18/18 [==============================] - 0s 2ms/step - loss: 374.5230 - accuracy: 0.4278
Epoch 6/30
18/18 [==============================] - 0s 2ms/step - loss: 186.9957 - accuracy: 0.4365
Epoch 7/30
18/18 [==============================] - 0s 2ms/step - loss: 118.6454 - accuracy: 0.4470
Epoch 8/30
18/18 [==============================] - 0s 2ms/step - loss: 163.6555 - accuracy: 0.4435
Epoch 9/30
18/18 [==============================] - 0s 2ms/step - loss: 98.9307 - accuracy: 0.4487
Epoch 10/30
18/18 [==============================] - 0s 2ms/step - loss: 72.0091 - accuracy: 0.4470
Epoch 1

In [42]:
# Evaluate the model on the validation set
best_model = grid_search.best_estimator_
result = best_model.predict_proba(X_val)
print("Validation Predictions:")
print(result)

7/7 [==============================] - 0s 1ms/step
Validation Predictions:
[[0.4013583  0.27171195 0.3269297 ]
 [0.4013583  0.27171195 0.3269297 ]
 [0.4013583  0.27171195 0.3269297 ]
 [0.4013583  0.27171195 0.3269297 ]
 [0.4013583  0.27171195 0.3269297 ]
 [0.4013583  0.27171195 0.3269297 ]
 [0.4013583  0.27171195 0.3269297 ]
 [0.4013583  0.27171195 0.3269297 ]
 [0.4013583  0.27171195 0.3269297 ]
 [0.4013583  0.27171195 0.3269297 ]
 [0.4013583  0.27171195 0.3269297 ]
 [0.4013583  0.27171195 0.3269297 ]
 [0.4013583  0.27171195 0.3269297 ]
 [0.4013583  0.27171195 0.3269297 ]
 [0.4013583  0.27171195 0.3269297 ]
 [0.4013583  0.27171195 0.3269297 ]
 [0.4013583  0.27171195 0.3269297 ]
 [0.4013583  0.27171195 0.3269297 ]
 [0.4013583  0.27171195 0.3269297 ]
 [0.4013583  0.27171195 0.3269297 ]
 [0.4013583  0.27171195 0.3269297 ]
 [0.4013583  0.27171195 0.3269297 ]
 [0.4013583  0.27171195 0.3269297 ]
 [0.4013583  0.27171195 0.3269297 ]
 [0.4013583  0.27171195 0.3269297 ]
 [0.4013583  0.27171195 0

In [43]:
max(result[0])

0.4013583

In [44]:
y_val

,label_1,label_X,label_2
0,0,0,1
1,0,1,0
2,0,1,0
3,1,0,0
4,0,0,1
...,...,...,...
211,0,0,1
212,0,0,1
213,1,0,0
214,0,1,0


In [50]:
c = 0
for i in range(0, y_val.shape[0]):
    r = np.where(result[i][0] == max(result[i]), "1", np.where(result[i][2] == max(result[2]), "2", "X"))
    c = c + np.where(((r == "1") & (y_val["label_1"][i] == 1)) | ((r == "X") & (y_val["label_X"][i] == 1)) | ((r == "2") & (y_val["label_2"][i] == 1)), 1, 0)

print("Accuracy: ", c / y_val.shape[0])

Accuracy:  0.4027777777777778
